In [237]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False



# 행정동 법정동 주소 맵핑 코드 (구주소 포함X)

* 행정동으로 전환 : mapping_h
* 법정동으로 전환 : mapping_b

## ======사용방식=====

*  기초통계자료만 활용한다.
*  해당통계표에 merge를 사용하돼, 입력코드와 입력명 열을 기준으로 사용할 것
*  과거지표 사용시에 시도 시군구 입력명 열을 기준으로 사용할 것

In [200]:
# 주소 맵핑 코드

# 문제점 : 행정동법정동 주소 혼용시 기준을 명확히 할 수 없고, 잦은 변경으로 인해 불편함.


# 모든 주소를 받아서 행정동으로 일괄적으로 변경 가능한 코드를 제작해야 통계집계시 용이할 것으로 판단
# 구질구질한 집계치나 세종특별시 같은 결측치 대치 후
# 법정동과 행정동이 중복되는 행을 삭제해야한다.



# ======================================= mapping =======================================


# 최신주소 : 행안부 지방행정실 주민등록업무 행정기관 및 관할구역 변경내역 참조
mapping = pd.read_excel('./dataset/mapping.xlsx')




# mapping.info()
mapping.drop(['생성일자','말소일자'],axis=1, inplace=True)
# mapping[mapping.isnull()]

# mapping.loc[(mapping['시군구명'].isnull())&(mapping['읍면동명'].isnull())]
# mapping.loc[(mapping['시군구명'].isnull())&(mapping['읍면동명'].isnull())].index
I = mapping.loc[(mapping['시군구명'].isnull())&(mapping['읍면동명'].isnull())].index
mapping.drop(I,inplace=True) # 제거완료

# mapping[mapping.읍면동명.isnull()]
# mapping.isnull().sum()

mapping['시군구명'] = mapping.apply(lambda x: x['시도명'] if x['시도명']=='세종특별자치시' else x['시군구명'], axis=1)

# mapping[mapping.시군구명==mapping.동리명]
I = mapping[mapping.시군구명==mapping.동리명].index
mapping.drop(I,inplace =True)

# mapping[mapping['읍면동명'].isnull()]
I = mapping[mapping['읍면동명'].isnull()].index
mapping = mapping.drop(I)

# mapping.isnull().sum()

mapping['시도코드'] = mapping['행정동코드'].apply(lambda x: str(x)[0:2])
mapping['시군구코드'] = mapping['행정동코드'].apply(lambda x: str(x)[0:5])

mapping = mapping[['시도코드','시도명','시군구코드','시군구명','행정동코드','읍면동명','법정동코드','동리명']].reset_index(drop =True)
# 읍면동명은 행정동명 , 동리명은 법정동명으로!
mapping.columns = ['시도코드','시도명','시군구코드','시군구명','행정동코드','행정동명','법정동코드','법정동명']



mapping


# ======================================= mapping_h =======================================
# 행정동 법정동을 행정동로 전환



# mapping[mapping['행정동명']==mapping['법정동명']]
# mapping[mapping['행정동코드']==mapping['법정동코드']]
# mapping[(mapping['행정동명']==mapping['법정동명'])&(mapping['행정동코드']==mapping['법정동코드'])]


mapping_h1 = mapping[['시도코드','시도명','시군구코드','시군구명','행정동코드','행정동명','행정동코드','행정동명']]
mapping_h1.columns = ['시도코드','시도명','시군구코드','시군구명','입력코드','입력명','행정동코드','행정동명']

mapping_h2 = mapping[['시도코드','시도명','시군구코드','시군구명','법정동코드','법정동명','행정동코드','행정동명']]
mapping_h2.columns = ['시도코드','시도명','시군구코드','시군구명','입력코드','입력명','행정동코드','행정동명']

mapping_h = pd.concat([mapping_h1,mapping_h2], join='outer').reset_index(drop=True)

# mapping_h
# len(mapping_h)
mapping_h = mapping_h.drop_duplicates().sort_values('행정동코드').reset_index(drop= True) 


mapping_h


# ======================================= mapping_b =======================================
# 행정동 법정동을 법정동로 전환



mapping_b1 = mapping[['시도코드','시도명','시군구코드','시군구명','행정동코드','행정동명','법정동코드','법정동명']]
mapping_b1.columns = ['시도코드','시도명','시군구코드','시군구명','입력코드','입력명', '법정동코드','법정동명']

mapping_b2 = mapping[['시도코드','시도명','시군구코드','시군구명','법정동코드','법정동명','법정동코드','법정동명']]
mapping_b2.columns = ['시도코드','시도명','시군구코드','시군구명','입력코드','입력명','법정동코드','법정동명']

mapping_b = pd.concat([mapping_b1,mapping_b2], join='outer').reset_index(drop=True)
mapping_b = mapping_b.drop_duplicates().sort_values('법정동코드').reset_index(drop= True) 


mapping_b




,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명
0,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010100,청운동
1,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010200,신교동
2,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010300,궁정동
3,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010400,효자동
4,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010500,창성동
...,...,...,...,...,...,...,...,...
21408,50,제주특별자치도,50130,서귀포시,5013061000,중문동,5013011400,대포동
21409,50,제주특별자치도,50130,서귀포시,5013061000,중문동,5013011800,하원동
21410,50,제주특별자치도,50130,서귀포시,5013062000,예래동,5013011900,색달동
21411,50,제주특별자치도,50130,서귀포시,5013062000,예래동,5013012000,상예동


,시도코드,시도명,시군구코드,시군구명,입력코드,입력명,행정동코드,행정동명
0,11,서울특별시,11110,종로구,1111051500,청운효자동,1111051500,청운효자동
1,11,서울특별시,11110,종로구,1111010300,궁정동,1111051500,청운효자동
2,11,서울특별시,11110,종로구,1111010400,효자동,1111051500,청운효자동
3,11,서울특별시,11110,종로구,1111010500,창성동,1111051500,청운효자동
4,11,서울특별시,11110,종로구,1111010800,통인동,1111051500,청운효자동
...,...,...,...,...,...,...,...,...
23527,50,제주특별자치도,50130,서귀포시,5013011200,중문동,5013061000,중문동
23528,50,제주특별자치도,50130,서귀포시,5013062000,예래동,5013062000,예래동
23529,50,제주특별자치도,50130,서귀포시,5013012000,상예동,5013062000,예래동
23530,50,제주특별자치도,50130,서귀포시,5013011900,색달동,5013062000,예래동


,시도코드,시도명,시군구코드,시군구명,입력코드,입력명,법정동코드,법정동명
0,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010100,청운동
1,11,서울특별시,11110,종로구,1111010100,청운동,1111010100,청운동
2,11,서울특별시,11110,종로구,1111010200,신교동,1111010200,신교동
3,11,서울특별시,11110,종로구,1111051500,청운효자동,1111010200,신교동
4,11,서울특별시,11110,종로구,1111010300,궁정동,1111010300,궁정동
...,...,...,...,...,...,...,...,...
40280,50,제주특별자치도,50130,서귀포시,5013032000,표선면,5013032024,가시리
40281,50,제주특별자치도,50130,서귀포시,5013032025,세화리,5013032025,세화리
40282,50,제주특별자치도,50130,서귀포시,5013032000,표선면,5013032025,세화리
40283,50,제주특별자치도,50130,서귀포시,5013032026,토산리,5013032026,토산리


# ====================================================